# Aprendizado Federado para reconhecimento de contexto em dispositivos móveis
## Experimento 1 - Pré-treinamento usual com uma base de usuários e Aprendizado Federado embarcado partindo do modelo pretreinado 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import es_utils as utils

Inicialmente precisamos separar quais 'labels' usaremos. Aqui, optamos pelas 51 labels que segundo artigo do ExtraSensosy utiliza, de modo a ter bons parâmetros de comparação

In [2]:
labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)

51

Separamos os 60 usuários em 5 pastas onde cada pasta considera 40 usuários selecionados aleatoriamente para treino e os 20 restantes para teste. Os dados dos 20 usuários são ainda separados em 4 arquivos *.csv* (x_train, x_test, y_train, y_test) e colocados em pastas nomeadas com o uuid do usuário.

In [3]:
#k_folds = utils.create_k_folds_n_users(5, 3, '/home/wander/OtherProjects/har_flower/sample_data')
k_folds = utils.create_k_folds_n_users(5, 40, '/home/wander/OtherProjects/har_flower/full_data')

Em seguida, para cada pasta, treinamos um modelo de MLP fazendo uma otimização de hiperparâmetros e convertemos em um modelo *tflite*. Os hiperparâmetros otimizados são:

- número de camadas escondidas (*hidden layers*): 1 ou 2
- número de neurônios em cada camada escondida: 4 a 64, com um passo de tamanho 4
- constante de penalização da regularização L2 em cada camada: 1e-2, 1e-3 ou 1e-4
- taxa de aprendizado (*learning rate*) do otimizador (*Stochastic Gradient Descent*, **SGD**): 1e-1, 1e-2, 1e-3 ou 1e-4
- momentum do otimizador (**SGD**): 0.8, 0.5, 0.3 ou 0.1

In [4]:

for fold in k_folds:
    config = {'df': k_folds[fold]['train']}
    har = utils.HAR(config)
    #har.run()
    har.hypertunning()
    
    har.mlp.model.save(f'model/saved_model_{fold}')
    converter = tf.lite.TFLiteConverter.from_saved_model(f'model/saved_model_fold{fold}') # path to the SavedModel directory
    tflite_model = converter.convert()
    with open(f'model/model_base_{fold}.tflite', 'wb') as f:
      f.write(tflite_model)

INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1/oracle.json
INFO:tensorflow:Reloading Tuner from hypertunning/experimento_1/tuner0.json
INFO:tensorflow:Oracle triggered exit


2022-03-17 16:39:54.734930: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-17 16:39:54.816943: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-17 16:39:54.816960: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-17 16:39:54.817550: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Epoch 1/50
3359/3359 [==============================] - 3s 744us/step - loss: 0.3222 - categorical_accuracy: 0.7201 - val_loss: 0.3145 - val_categorical_accuracy: 0.7252
Epoch 2/50
3359/3359 [==============================] - 2s 695us/step - loss: 0.3100 - categorical_accuracy: 0.7300 - val_loss: 0.3105 - val_categorical_accuracy: 0.7273
Epoch 3/50
3359/3359 [==============================] - 2s 694us/step - loss: 0.3071 - categorical_accuracy: 0.7321 - val_loss: 0.3087 - val_categorical_accuracy: 0.7284
Epoch 4/50
3359/3359 [==============================] - 2s 693us/step - loss: 0.3055 - categorical_accuracy: 0.7338 - val_loss: 0.3073 - val_categorical_accuracy: 0.7321
Epoch 5/50
3359/3359 [==============================] - 2s 694us/step - loss: 0.3044 - categorical_accuracy: 0.7351 - val_loss: 0.3064 - val_categorical_accuracy: 0.7322
Epoch 6/50
3359/3359 [==============================] - 2s 695us/step - loss: 0.3035 - categorical_accuracy: 0.7360 - val_loss: 0.3057 - val_categoric

Epoch 49/50
3359/3359 [==============================] - 2s 688us/step - loss: 0.2959 - categorical_accuracy: 0.7425 - val_loss: 0.2985 - val_categorical_accuracy: 0.7405
Epoch 50/50
3359/3359 [==============================] - 2s 684us/step - loss: 0.2959 - categorical_accuracy: 0.7426 - val_loss: 0.2985 - val_categorical_accuracy: 0.7401
Best epoch: 49


2022-03-17 16:39:55.045964: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 195198208 exceeds 10% of free system memory.
2022-03-17 16:41:51.867360: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 195198208 exceeds 10% of free system memory.


...

INFO:tensorflow:Oracle triggered exit


Epoch 1/50
3749/3749 [==============================] - 3s 763us/step - loss: 0.3300 - categorical_accuracy: 0.7140 - val_loss: 0.3197 - val_categorical_accuracy: 0.7231
Epoch 2/50
3749/3749 [==============================] - 3s 726us/step - loss: 0.3174 - categorical_accuracy: 0.7232 - val_loss: 0.3153 - val_categorical_accuracy: 0.7262
Epoch 3/50
3749/3749 [==============================] - 3s 727us/step - loss: 0.3144 - categorical_accuracy: 0.7259 - val_loss: 0.3130 - val_categorical_accuracy: 0.7271
Epoch 4/50
3749/3749 [==============================] - 3s 727us/step - loss: 0.3126 - categorical_accuracy: 0.7267 - val_loss: 0.3116 - val_categorical_accuracy: 0.7281
Epoch 5/50
3749/3749 [==============================] - 3s 727us/step - loss: 0.3114 - categorical_accuracy: 0.7273 - val_loss: 0.3107 - val_categorical_accuracy: 0.7290
Epoch 6/50
3749/3749 [==============================] - 3s 728us/step - loss: 0.3105 - categorical_accuracy: 0.7283 - val_loss: 0.3099 - val_categoric

Epoch 49/50
3749/3749 [==============================] - 3s 724us/step - loss: 0.3023 - categorical_accuracy: 0.7348 - val_loss: 0.3020 - val_categorical_accuracy: 0.7339
Epoch 50/50
3749/3749 [==============================] - 3s 728us/step - loss: 0.3023 - categorical_accuracy: 0.7348 - val_loss: 0.3020 - val_categorical_accuracy: 0.7340
Best epoch: 47
Epoch 1/47
11995/11995 [==============================] - 8s 686us/step - loss: 0.3250 - categorical_accuracy: 0.7172 - val_loss: 0.3174 - val_categorical_accuracy: 0.7238
Epoch 2/47
11995/11995 [==============================] - 8s 677us/step - loss: 0.3151 - categorical_accuracy: 0.7241 - val_loss: 0.3132 - val_categorical_accuracy: 0.7266
Epoch 3/47
11995/11995 [==============================] - 8s 678us/step - loss: 0.3127 - categorical_accuracy: 0.7259 - val_loss: 0.3115 - val_categorical_accuracy: 0.7268
Epoch 4/47
11995/11995 [==============================] - 8s 674us/step - loss: 0.3112 - categorical_accuracy: 0.7268 - val_los

11995/11995 [==============================] - 8s 687us/step - loss: 0.3029 - categorical_accuracy: 0.7336 - val_loss: 0.3025 - val_categorical_accuracy: 0.7337
Epoch 47/47
1562/1562 [==============================] - 1s 480us/step - loss: 0.3045 - categorical_accuracy: 0.7310
4
Test results - Loss: 0.3044818639755249 - Accuracy: 0.7310216426849365%
Averaged Balanced Accuracy: 0.716486
INFO:tensorflow:Assets written to: model/saved_model_foldfold_4/assets


INFO:tensorflow:Assets written to: model/saved_model_foldfold_4/assets
2022-03-17 17:22:13.303266: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-03-17 17:22:13.303285: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-03-17 17:22:13.303290: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-03-17 17:22:13.303465: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/saved_model_foldfold_4
2022-03-17 17:22:13.304361: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-03-17 17:22:13.304373: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: model/saved_model_foldfold_4
2022-03-17 17:22:13.306924: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-03-17 17:22:13.330027: I tensorflow/cc/saved_m

Com os modelos salvos, o restante do experimento segue em ambiente Android